In [27]:
import pandas as pd
import folium

# Read in the GTFS files as DataFrames
agency = pd.read_csv('agency.txt')
stops = pd.read_csv('stops.txt')
routes = pd.read_csv('routes.txt')
trips = pd.read_csv('trips.txt')
stop_times = pd.read_csv('stop_times.txt')

# Merge the DataFrames to create a complete dataset
merged = pd.merge(pd.merge(pd.merge(agency, routes, on='agency_id'), trips, on='route_id'), stop_times, on='trip_id')
merged = pd.merge(merged, stops, on='stop_id')

# Group the dataset by route and direction
grouped = merged.groupby(['route_id', 'direction_id'])

# Calculate the length of each route and select the longest one
route_lengths = grouped.apply(lambda x: x['shape_dist_traveled'].max() - x['shape_dist_traveled'].min())
longest_route_id = route_lengths.idxmax()[0]

# Filter the dataset to only include the longest route
longest_route = merged[merged['route_id'] == longest_route_id]

# Create a map centered on the first stop in the longest route
map_center = (longest_route['stop_lat'].iloc[0], longest_route['stop_lon'].iloc[0])
m = folium.Map(location=map_center, zoom_start=12)

# Extract the stop locations and create a line between them for the longest route
locations = longest_route[['stop_lat', 'stop_lon']].values.tolist()
folium.PolyLine(locations=locations, color='blue', weight=2.5, opacity=1, tooltip=f'Route {longest_route_id}').add_to(m)

# Display the map with a description tag for the longest route
m.get_root().html.add_child(folium.Element(f'<h3>Longest Route: {longest_route_id}</h3>'))
m

